# Author: Tobias

In [4]:
import os 
import datetime
import json
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import tensorflow as tf

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())
basepath = Path(os.getcwd())

%load_ext autoreload
%autoreload 2
basepath

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


PosixPath('/srv/idp-radio-1')

In [5]:
# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Or 2, 3, etc. other than 0

#config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 1.2
tf.compat.v1.Session(config=config)
tf.__version__

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:03:00.0, compute capability: 7.5



'2.3.0'

In [6]:
import numpy as np

import traceback
from sklearn.metrics import classification_report
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam, SGD
from keras.utils.generic_utils import get_custom_objects

from tensorflow.keras.applications import InceptionV3, Xception, DenseNet121, InceptionResNetV2, ResNet152V2, NASNetLarge
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.simple.load_model import *
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.benchmarks.benchmark_definitions import generate_benchmarks,simple_architecture_experiment, Chexpert_Benchmark, CHEXPERT_COLUMNS, METRICS, SINGLE_CLASS_METRICS
from src.metrics.metrics import F2Score, SingleClassMetric
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

In [7]:
experiments = get_all_experiment_logs()
experiments = [exp for exp in experiments if "Failed" not in exp["name"]]
#experiments = [exp for exp in experiments if "N12" in exp["name"]]
experiments = [exp for exp in experiments if "num_samples_test" in exp["benchmark"].keys() ]
experiments = [exp for exp in experiments if exp["benchmark"]["num_samples_test"] == 234 ]
len(experiments)

40

In [8]:
exp_dict = experiments[-2].copy()
exp_dict["name"], exp_dict["test"]["auc"]

('DenseNet121_Chexpert_BCE_E3_B32_C0_N12_AugAffine_Uones_D256_DS9505_2LR1_LF5_SGD_Upsampled',
 0.7735175490379333)

In [22]:
print("Reevaluate model {} - {} ".format(exp_dict["name"], exp_dict["id"]), "\n")
exp = rebuild_experiment(exp_dict)
exp_dict["test_again"] = reevaluate(exp, new_metrics=True)
print(exp_dict["test"]["auc"] - exp_dict["test_again"]["auc"])
print("sum difference", sum(difference_test_results(exp_dict["test"], exp_dict["test_again"]).values()))

Reevaluate model DenseNet121_Chexpert_BCE_E3_B32_C0_N12_AugAffine_Uones_D256_DS9505_2LR1_LF5_SGD_Upsampled - 03479e56-ee90-11ea-b021-0242ac110002  

Using weights file DenseNet121_Chexpert_BCE_E3_B32_C0_N12_AugAffine_Uones_D256_DS9505_2LR1_LF5_SGD_Upsampled_20200904-092128.h5 to load model...
234/234 [==============================] - 4s 18ms/step
-0.1029731035232544
sum difference 0.1029731035232544


In [6]:
#exp_dict["test_again"]

In [11]:
new_experiments = []
for index, exp_dict in enumerate(experiments[:]):
    for epoch_id in range(exp_dict["benchmark"]["epochs"]):
        new_exp_dict = exp_dict.copy()
        new_exp_dict["epoch_model"] = epoch_id + 1
        try:
            print("\nReevaluate model {} - {} ".format(index, new_exp_dict["id"]))
            print("For Epoch {} ".format(new_exp_dict["epoch_model"]))
            
            exp = rebuild_experiment(exp_dict, epoch=new_exp_dict["epoch_model"])
            new_exp_dict["test_again"] = reevaluate(exp, new_metrics=True)
            new_exp_dict["val_again"] = reevaluate_validation(exp, new_metrics=True)
            new_exp_dict["val"] = {name.replace("val_", ""):res[-1] for name, res in new_exp_dict["history"].items() if name.startswith("val_")}

            new_experiments.append(new_exp_dict)

            print("test_again auc is ", new_exp_dict["test_again"]["auc"])
            print("test auc is ", new_exp_dict["test"]["auc"])
            print("val_again auc is ", new_exp_dict["val_again"]["auc"])
            print("val auc is ", new_exp_dict["val"]["auc"])

            del exp
        except:
            print(traceback.format_exc())
        


Reevaluate model 0 - 06bbf24c-e0cd-11ea-b9dc-0242ac110003 
For Epoch 1 
Using weights file weights.01-0.35.hdf5 to load model...
11596/11596 [==============================] - 220s 19ms/step
test_again auc is  0.82482755
test auc is  0.8351618051528931
val_again auc is  0.8549509
val auc is  0.7993749976158142

Reevaluate model 0 - 06bbf24c-e0cd-11ea-b9dc-0242ac110003 
For Epoch 2 
Using weights file DenseNet121_Chexpert_BCE_E5_B32_C0_N5_D256_DS9505_LR4_20200817-210257.h5 to load model...
11596/11596 [==============================] - 221s 19ms/step
test_again auc is  0.8034682
test auc is  0.8351618051528931
val_again auc is  0.8492519
val auc is  0.7993749976158142

Reevaluate model 0 - 06bbf24c-e0cd-11ea-b9dc-0242ac110003 
For Epoch 3 
Using weights file DenseNet121_Chexpert_BCE_E5_B32_C0_N5_D256_DS9505_LR4_20200817-210257.h5 to load model...
11596/11596 [==============================] - 221s 19ms/step
test_again auc is  0.8034682
test auc is  0.8351618051528931
val_again auc is  

In [16]:
[{
    "name":new_experiment["name"], 
    "id":new_experiment["id"],
    "epoch": new_experiment["epoch_model"],
    "test auc":new_experiment["test"]["auc"],
    "test again auc":new_experiment["test_again"]["auc"],
    "val auc":new_experiment["val"]["auc"] if "auc" in new_experiment["val"].keys() else None, 
    "val again auc": new_experiment["val_again"]["auc"]
 }
    for new_experiment in new_experiments]

[{'name': 'DenseNet121_Chexpert_BCE_E5_B32_C0_N5_D256_DS9505_LR4',
  'id': '06bbf24c-e0cd-11ea-b9dc-0242ac110003',
  'epoch': 1,
  'test auc': 0.8351618051528931,
  'test again auc': 0.82482755,
  'val auc': 0.7993749976158142,
  'val again auc': 0.8549509},
 {'name': 'DenseNet121_Chexpert_BCE_E5_B32_C0_N5_D256_DS9505_LR4',
  'id': '06bbf24c-e0cd-11ea-b9dc-0242ac110003',
  'epoch': 2,
  'test auc': 0.8351618051528931,
  'test again auc': 0.8034682,
  'val auc': 0.7993749976158142,
  'val again auc': 0.8492519},
 {'name': 'DenseNet121_Chexpert_BCE_E5_B32_C0_N5_D256_DS9505_LR4',
  'id': '06bbf24c-e0cd-11ea-b9dc-0242ac110003',
  'epoch': 3,
  'test auc': 0.8351618051528931,
  'test again auc': 0.8034682,
  'val auc': 0.7993749976158142,
  'val again auc': 0.8492519},
 {'name': 'DenseNet121_Chexpert_BCE_E5_B32_C0_N5_D256_DS9505_LR4',
  'id': '06bbf24c-e0cd-11ea-b9dc-0242ac110003',
  'epoch': 4,
  'test auc': 0.8351618051528931,
  'test again auc': 0.8330408,
  'val auc': 0.7993749976158142

In [ ]:
[new_experiment["test_again"]["auc"] for new_experiment in new_experiments], [new_experiment["test"]["auc"] for new_experiment in new_experiments]

In [32]:
# cast from np.float32 to float:
for exp in new_experiments:
    exp["test_again"] = {k:float(v) for k, v in exp["test_again"].items()}
    exp["val_again"] = {k:float(v) for k, v in exp["val_again"].items()}

output_file = Path("/srv/idp-radio-1/logs/epoch_reevaluation_experiments.json")
with open(output_file, 'w') as f:
    data = {"experiments":new_experiments}
    json_data = json.dumps(data, indent=4)
    f.write(json_data)

In [28]:
for exp in new_experiments:
    classes = [classname.lower().replace(" ", "_") for classname in exp["benchmark"]["label_columns"]]
    if len(classes) == 12:
        #print([exp["test_again"]["auc_"+classname]for classname in classes])
        exp["test_again"]["auc"] = sum([exp["test_again"]["auc_"+classname]for classname in classes])/(len(classes)-1)

In [31]:
res = [(new_experiment["id"], new_experiment["epoch_model"], new_experiment["test_again"]["auc"]) for new_experiment in new_experiments if "N12" in new_experiment["name"]]
res.sort(key=lambda tup: tup[2])
res.reverse()
res



[('bd5c2e9a-e9ec-11ea-ae3a-0242ac110005', 3, 0.8613448359749534),
 ('bd5c2e9a-e9ec-11ea-ae3a-0242ac110005', 4, 0.8604057214476846),
 ('bd5c2e9a-e9ec-11ea-ae3a-0242ac110005', 5, 0.8603749329393561),
 ('bd5c2e9a-e9ec-11ea-ae3a-0242ac110005', 2, 0.86009220643477),
 ('bca75ad8-e6f4-11ea-a90b-0242ac110005', 2, 0.8534599163315513),
 ('bca75ad8-e6f4-11ea-a90b-0242ac110005', 4, 0.8475377505475824),
 ('74d8220a-e6c2-11ea-b084-0242ac110005', 2, 0.8475213592702692),
 ('bca75ad8-e6f4-11ea-a90b-0242ac110005', 5, 0.8474861112507907),
 ('1262eecc-e490-11ea-a3de-0242ac110005', 3, 0.8471921357241544),
 ('bca75ad8-e6f4-11ea-a90b-0242ac110005', 3, 0.8467668078162454),
 ('74d8220a-e6c2-11ea-b084-0242ac110005', 1, 0.8463928970423612),
 ('3ecd1de0-e619-11ea-8544-0242ac110005', 3, 0.8454478274692189),
 ('5663f98e-e44c-11ea-bed5-0242ac110005', 3, 0.8447984944690358),
 ('03479e56-ee90-11ea-b021-0242ac110002', 3, 0.8438373370604082),
 ('74d8220a-e6c2-11ea-b084-0242ac110005', 4, 0.8436806310306896),
 ('74d8220a-

In [ ]:
exp.benchmark.metrics